W tym przykładzie mamy doczynienia z generowaniem  ciągu (np. muzyki).  
I tak jak podczas prezentacaji przedstawiłem model Transformera, tak tutaj będzie on wyglądał trochę inaczej.
W tym przypadku bowiem nie interesuje nas model seq2seq (np. jak przy translacji),
a model zdolny do generowania nowej muzyki.  
Zatem z niczego (a w typ przypadku z kilku początkowych inputów, nut) mamy stworzyć pełną sekwencję.
Dla przypomnienia klasyczny model Transformera.

<img src="https://www.researchgate.net/publication/323904682/figure/fig1/AS:606458626465792@1521602412057/The-Transformer-model-architecture.png" width=300 height=300 />

Dla generowania sekwencji nie jest potrzebna cała lewa strona, która jest Encoderem.
W normalnym przypadku odpowiadałaby ona za "wyciągnięcie" pewnych informacji z ciągu wejściowego, lecz tutaj zwyczajnie go nie ma :)
Zostaje więc strona prawa, czyli Decoder.
Jest on odpowiedzialny za zakodowanie informacji, którą już zdobyliśmy (np. początek zdania w języku na który tłumaczymy) i wyciągnięcie z niej kolejnego wyrazu.

Jak to działa?

Na początku brany jest cały ciąg dotychczas wygenerowany i przepuszczany jest przez warstwę Positional Encoding, która dodaje informację na których miejscach znajdują się dane słowa (niektóre wagi połączeń są dzielone, więc trzeba dodatkową informację o pozycji).

Następnie tak przygotowany ciąg przechodzi przez warstwę Masked Multi-Head Attention.  
Dlaczego Masked?  
Ponieważ dołożona jest "maska", która daje znać sieci, które miejsca w ciągu są wypełnione, a które są "puste" - czyli całkowicie do pominięcia przy obliczeniach.  
Warto zauważyć, że do tej warstwy, wchodzą 3 wejścia i reprezentują one Key, Query i Value, a wszystkie one pochodzą z poprzedniej wartwy (dlatego jest to self-attention).

Kolejna warstwa Add & Norm zbiera wyjście z Multi-Head Attention oraz oryginalny ciąg przed wejściem do tej warstwy. Wektory te odpowiednio dodaje i normalizuje.

W klasycznym Transformerze teraz następowałoby połączenia danych z Encodera (wtedy Key i Values w mechanizmie attention pochodziłyby z oryginalnego ciągu. Przy generowaniu muzyki nie ma takiej potrzeby, więc znowu może to być self-attention.

Znowu występuje warstwa Add & Norm, po czym jest warstwa feedforward i kolejna Add & Norm.

Co ważne taka konfiguracja jest powtarzana kilka razy!

Na samym końcu znajduje się warstwa liniowa i softmax w celu zebrania wyniku.

Chciałbym się tutaj skupić właśnie na mechanizmie self-attention, ponieważ reszta jest pewnie znajoma.

Dla podsumowania.
Mamy ciąg np. słów, zapisanych w postaci wektorów.  
Po zastosowaniu na nim mechanizmu attention otrzymujemy nową macierz, która jest reprezentacją tamtego ciągu.  
W transformerze operację taką wykonujemy wiele razy, na końcu decydując jakie słowo powinno wystąpić kolejne.

Jak wygląda obliczanie attention (na podstawie  
https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a  
https://medium.com/lsc-psd/introduction-of-self-attention-layer-in-transformer-fc7bff63f3bc):  
Założenie:  
Wejście: input_1, input_2, input_3 to kolejne słowa w zdaniu.  
  
1. Przygotowanie inputów.
2. Inicjalizacja wag macierzy Wk, Wq, Wv (przygotowanie).
3. Obliczenie key, query i value.
4. Obliczenie wartości attention dla inputu_1.
5. Obliczenie funkcji softmax.
6. Pomnożenie tych wartości z values.
7. Zsumowanie tych wartości i otrzymanie Outputu 1.
8. Powtórzenie kroków 4–7 dla Inputu 2 i 3 (kroki te można wykonać równolegle poprzez odpowiednie zarządzanie macierzami).

In [1]:
# Krok 1: te "inputy" to kolejne np słowa w ciągu, zakodowane jako wektory
import numpy as np
input_1 = np.array([1, 0, 1, 0])
input_2 = np.array([0, 2, 0, 2])
input_3 = np.array([1, 1, 1, 1])

input = np.array([input_1, input_2, input_3])  # w postaci macierzowej
input

array([[1, 0, 1, 0],
       [0, 2, 0, 2],
       [1, 1, 1, 1]])

In [2]:
# Krok 2
# Każdy input ma swoje wartości odpowiadających mu wektorów w postaci key, query, value.
# Aby otrzymać te wartości input przepuszczany jest przez warstwę w pełni połączonych neuronów 
# (czyli generalnie wykonywane jest mnożenie przez macierz - każda z nich (Wk, Wq, Wv ma swoje wagi)).
# Wagi te są na początku inicjalizowane małymi liczbmi, a następnie sieć się ich uczy. 
# W tym przykładzie rozmiar key, query i value będzie wynosił 3 (rozmiar inputu to 4).

Wk = np.array(  [[0, 0, 1],
                 [1, 1, 0],
                 [0, 1, 0],
                 [1, 1, 0]])

Wq = np.array(  [[1, 0, 1],
                 [1, 0, 0],
                 [0, 0, 1],
                 [0, 1, 1]])

Wv = np.array(  [[0, 2, 0],
                 [0, 3, 0],
                 [1, 0, 3],
                 [1, 1, 0]])
Wk

array([[0, 0, 1],
       [1, 1, 0],
       [0, 1, 0],
       [1, 1, 0]])

In [4]:
#Krok 3
# Obliczenie wartości key, query i value dla inputu_1.

#                [0, 0, 1]
# [1, 0, 1, 0] x [1, 1, 0] = [0, 1, 1]
#                [0, 1, 0]
#                [1, 1, 0]
key_1 = input_1.dot(Wk)
key_1

array([0, 1, 1])

In [38]:
# Obliczanie dla wszystkich inputów
key_1 = input_1.dot(Wk)
query_1 = input_1.dot(Wq)
value_1 = input_1.dot(Wv)

# ...
# w skrócie w postaci macierzowej
key_matrix = input.dot(Wk)
query_matrix = input.dot(Wq)
value_matrix = input.dot(Wv)

print(f'Key for input 1: \n{key_1}\n')
print(f'Key: \n{key_matrix}')
print(f'Query: \n{query_matrix}')
print(f'Value: \n{value_matrix}')

Key for input 1: 
[0 1 1]

Key: 
[[0 1 1]
 [4 4 0]
 [2 3 1]]
Query: 
[[1 0 2]
 [2 2 2]
 [2 1 3]]
Value: 
[[1 2 3]
 [2 8 0]
 [2 6 3]]


<img src="https://miro.medium.com/max/1400/1*Cfsh9uK8Y6FhamziJZIKRA.jpeg" width=600 height=500 />

In [49]:
# Krok 4: Wyliczenie wartości attention dla input_1
# Sprawdzamy jak blisko query zgadza się z każdym key. Najprostszym sposobem jest iloczyn skalarny (dot product attention):
# (Na obrazku mamy dodatkowe skalowanie przez pierwiastek z wymiaru. Nie zgadzają się też wymiary.)
attention_score_1 = query_1.dot(key_matrix)
attention_score_1

array([4, 7, 3])

<img src="https://miro.medium.com/max/1400/1*4Ky7WD2Bwt7ONuewCEimbg.gif" width=600 height=300 />

In [54]:
# Krok 5: Skalowanie za pomocą funkcji softmax
from scipy.special import softmax
soft = softmax(attention_score_1)
soft
# Widać że input_1 jest mocno zależny od input_2, ponieważ warość attention_score jest dużo większa niż dla reszty

array([0.04661262, 0.93623955, 0.01714783])

W ten sposób obliczone została zgodność/zależność input_1 od reszty ciągu.
Jest to również waga z jaką odpowiadające im wartości wejdą do rezultatu.

In [62]:
# Krok 6: Pomnożenie otrzymanych wag z values
inp_1_val_1 = soft[0] * value_1
inp_1_val_2 = soft[1] * value_2
inp_1_val_3 = soft[2] * value_3
print(inp_1_val_1)
print(inp_1_val_2)
print(inp_1_val_3)
# w ten sposób otrzymaliśmy tablicę, która zawiera zsumowane inputy przeskalowane względem zgodności (softmax)
# poniższy wynik otrzymany został dla input_1

[0.04661262 0.09322525 0.13983787]
[1.8724791  7.48991642 0.        ]
[0.03429565 0.10288695 0.05144348]


In [65]:
# Krok 7: Zsumowanie tych wartości do jednego wektora
result = inp_1_val_1 + inp_1_val_2 + inp_1_val_3
result

array([1.95338738, 7.68602861, 0.19128134])

<img src="https://miro.medium.com/max/1400/1*tIU60poFU4Ym988ULlN1sA.gif" width=600 height=300 />

Krok 8
Powtórzenie obliczeń 4-7 dla inputu 2 i 3.

W ten sposób otrzymaliśmy nowy wektor dla input_1, który wziął pod uwagę inne inputy i stworzył całkiem nową reprezentacje ciągu. Podobne działania wykonujemy dla reszty inputów, w rezultacie otrzymując nową sekwencję.  
Jak zaznaczyłem przepuszczamy ją przez normalizację i uwaga:  
całość powtarzamy jeszcze kilka razy :D  
W ten sposób uzyskujemy wewnętrzną strukturę ciągu i reprezentację wiadomości jaka się za nim kryje.

<img src="https://miro.medium.com/max/1400/1*F2dNXYpvLwbqGLZtK0rFFQ.jpeg" width=600 height=300 />

Ale to jeszcze nie koniec :o  
Przedstawiłem tutaj zwykły mechanizm self-attention (bez self byłoby gdyby key i values, pochodziły z innego ciągu). Ale na rysunku Transformera widnieje "Multi-Head". Dlaczego?  
Aby poprawić działanie całego procesu, wejściową sekwencje przepuszczamy przez wiele różnych wag Wk, Wq, Wv za każdym razie otrzymując trochę co innego.  
Dopiero później je łączymy w jedną reprezentację i otrzymujemy wyjście z Multi-Head.  
A później normalizacja itp. i zaczynamy zabawę od początku.

<img src="https://miro.medium.com/max/1400/1*65w5woXDym6xClP8tqOifg.gif" width=600 height=300 />

Dlatego właśnie wyniki otrzymywane tą metodą są bardzo dobre - za pomocą wielu wag, możemy zwrócić szczególną uwagę na konkrenty zestaw wiadomości i później połączyć je w jedną.